# Day 8-6: Prophet을 활용한 비즈니스 시계열 예측 - 정답

---

## 🎯 실습 퀴즈 정답

In [ ]:
# 환경 설정
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import plotly.express as px
import plotly.graph_objects as go
import warnings

warnings.filterwarnings('ignore')

import logging
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

# 데이터 로드
sales_df = pd.read_csv('datasets/ecommerce_sales_data.csv', parse_dates=['ds'])
events_df = pd.read_csv('datasets/ecommerce_events.csv', parse_dates=['ds'])

# 사전 학습된 모델 (문제에서 사용)
df_prophet = sales_df[['ds', 'y']].copy()
model_with_holidays = Prophet(holidays=events_df, yearly_seasonality=True, weekly_seasonality=True)
model_with_holidays.fit(df_prophet)
future_holidays = model_with_holidays.make_future_dataframe(periods=30)
forecast_holidays = model_with_holidays.predict(future_holidays)

# 외부 변수 모델
df_with_regressor = sales_df[['ds', 'y', 'temperature', 'discount_rate']].copy()
model_regressor = Prophet(holidays=events_df, yearly_seasonality=True, weekly_seasonality=True)
model_regressor.add_regressor('temperature')
model_regressor.add_regressor('discount_rate')
model_regressor.fit(df_with_regressor)

print("환경 설정 완료!")

---

## Q1. Prophet 데이터 형식 ⭐

**문제**: 다음 데이터프레임을 Prophet 입력 형식으로 변환하세요.

In [ ]:
# Q1 정답
raw_df = pd.DataFrame({
    'order_date': pd.date_range('2024-01-01', periods=10),
    'revenue': [100, 120, 90, 150, 200, 180, 160, 140, 130, 170]
})

# Prophet 형식으로 변환: rename을 사용하여 컬럼명 변경
prophet_df = raw_df.rename(columns={
    'order_date': 'ds',  # 날짜 컬럼 -> ds
    'revenue': 'y'       # 타겟 컬럼 -> y
})

prophet_df

In [ ]:
# 테스트
assert 'ds' in prophet_df.columns, "ds 컬럼이 없습니다"
assert 'y' in prophet_df.columns, "y 컬럼이 없습니다"
assert pd.api.types.is_datetime64_any_dtype(prophet_df['ds']), "ds가 datetime이 아닙니다"
print("Q1 테스트 통과!")

### 💡 Q1 풀이 설명

**접근 방법**: Prophet은 엄격하게 `ds`, `y` 컬럼명을 요구합니다. `rename()` 함수를 사용하여 기존 컬럼명을 변환합니다.

**핵심 개념**:
- `ds`: datetime 타입의 날짜/시간 컬럼
- `y`: 예측하려는 수치형 타겟 컬럼

**대안 솔루션**:
```python
# 방법 2: 새 데이터프레임 생성
prophet_df = pd.DataFrame({'ds': raw_df['order_date'], 'y': raw_df['revenue']})

# 방법 3: copy 후 컬럼 할당
prophet_df = raw_df.copy()
prophet_df.columns = ['ds', 'y']
```

**흔한 실수**:
- 컬럼명을 `date`, `sales` 등으로 두고 그대로 사용 → Prophet 에러 발생
- datetime 변환 누락 (문자열 날짜)

**실무 팁**: 실무 데이터는 대부분 `date`, `timestamp`, `sales`, `revenue` 등의 이름을 사용하므로, Prophet 사용 전 반드시 컬럼명 확인 및 변환이 필요합니다.

---

## Q2. 기본 모델 학습 및 예측 ⭐

**문제**: Prophet 모델을 학습하고 7일 후까지 예측하세요.

In [ ]:
# Q2 정답
# 데이터 준비
train_df = sales_df[['ds', 'y']].copy()

# 모델 생성: 조건에 맞게 계절성 설정
model_q2 = Prophet(
    yearly_seasonality=True,   # 연간 계절성 활성화
    weekly_seasonality=True,   # 주간 계절성 활성화
    daily_seasonality=False    # 일간 계절성 비활성화 (일별 데이터이므로)
)

# 모델 학습
model_q2.fit(train_df)

# 미래 7일 예측을 위한 데이터프레임 생성
future_q2 = model_q2.make_future_dataframe(periods=7)

# 예측 수행
forecast_q2 = model_q2.predict(future_q2)

# 결과 확인
print(f"예측 기간: {forecast_q2['ds'].max().date()}까지")
forecast_q2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# 테스트
assert len(forecast_q2) == len(train_df) + 7, "예측 기간이 7일이 아닙니다"
assert 'yhat' in forecast_q2.columns, "yhat 컬럼이 없습니다"
print("Q2 테스트 통과!")

### 💡 Q2 풀이 설명

**접근 방법**: Prophet 워크플로우는 3단계입니다: (1) 모델 생성 → (2) fit → (3) predict

**핵심 개념**:
- `make_future_dataframe(periods=N)`: 기존 데이터 + 미래 N일의 날짜 생성
- `predict()`: 학습된 모델로 전체 기간 예측

**대안 솔루션**:
```python
# 주단위 예측 (freq='W')
future = model.make_future_dataframe(periods=1, freq='W')  # 1주
```

**흔한 실수**:
- `fit()` 호출 전에 `predict()` 호출 → 에러
- `periods=7`이 "7주"가 아닌 "7일"임을 혼동

**실무 팁**: `daily_seasonality=False`는 일별 데이터에서 권장됩니다. True로 설정하면 시간별 패턴을 찾으려 하여 과적합될 수 있습니다.

---

## Q3. 이벤트 효과 분석 ⭐⭐

**문제**: 블랙프라이데이의 평균 매출 기여도를 계산하세요.

In [ ]:
# Q3 정답
# 블랙프라이데이 효과 컬럼 확인
# holidays를 추가하면 각 이벤트명으로 컬럼이 생성됨
if 'black_friday' in forecast_holidays.columns:
    # 블랙프라이데이 효과가 0이 아닌 날만 필터링
    bf_days = forecast_holidays[forecast_holidays['black_friday'] != 0]
    
    # 평균 효과 계산
    bf_effect = bf_days['black_friday'].mean()
    
    print(f"블랙프라이데이 평균 매출 기여도: {bf_effect:,.0f}원")
    print(f"효과 적용 일수: {len(bf_days)}일")
else:
    print("black_friday 컬럼이 없습니다. holidays 파라미터를 확인하세요.")

In [ ]:
# 테스트
assert bf_effect > 0, "블랙프라이데이는 매출 상승 효과가 있어야 합니다"
print("Q3 테스트 통과!")

### 💡 Q3 풀이 설명

**접근 방법**: Prophet은 holidays 파라미터로 전달된 각 이벤트에 대해 별도의 컬럼을 생성합니다.

**핵심 개념**:
- 이벤트 효과 컬럼: 이벤트명과 동일한 이름의 컬럼
- 효과가 0인 날: 해당 이벤트가 적용되지 않은 날
- 평균 계산 시 0이 아닌 날만 필터링해야 정확

**대안 솔루션**:
```python
# 전체 이벤트 효과 한번에 계산
for event in events_df['holiday'].unique():
    if event in forecast_holidays.columns:
        effect = forecast_holidays[forecast_holidays[event] != 0][event].mean()
        print(f"{event}: {effect:,.0f}원")
```

**흔한 실수**:
- 전체 평균 계산 (0 포함) → 효과가 희석됨
- 이벤트명 오타 (대소문자 구분)

**실무 팁**: 이 분석 결과를 마케팅 팀에 보고할 때 "블랙프라이데이 순수 매출 기여도는 약 XXX만원"이라고 명확히 전달할 수 있습니다.

---

## Q4. 이벤트 윈도우 설정 ⭐⭐

**문제**: 조건에 맞는 이벤트 데이터프레임을 생성하세요.

In [ ]:
# Q4 정답
spring_promo = pd.DataFrame({
    'holiday': 'spring_promo',  # 이벤트명
    'ds': pd.to_datetime(['2024-03-01', '2024-03-15']),  # 이벤트 날짜들
    'lower_window': -2,  # 사전 효과 2일 (음수!)
    'upper_window': 5    # 사후 효과 5일
})

spring_promo

In [ ]:
# 테스트
assert len(spring_promo) == 2, "2개의 이벤트가 있어야 합니다"
assert spring_promo['lower_window'].iloc[0] == -2, "lower_window는 -2여야 합니다"
assert spring_promo['upper_window'].iloc[0] == 5, "upper_window는 5여야 합니다"
print("Q4 테스트 통과!")

### 💡 Q4 풀이 설명

**접근 방법**: Prophet의 holidays 데이터프레임은 4개의 필수 컬럼을 가집니다.

**핵심 개념**:
- `lower_window`: 이벤트 **이전** 영향 기간 (음수 또는 0)
- `upper_window`: 이벤트 **이후** 지속 기간 (0 이상)
- 예: `lower_window=-2, upper_window=5` → 이벤트일 기준 D-2 ~ D+5 (총 8일)

**대안 솔루션**:
```python
# 각 이벤트 별도 정의 후 concat
promo1 = pd.DataFrame({'holiday': 'spring_promo', 'ds': ['2024-03-01'], ...})
promo2 = pd.DataFrame({'holiday': 'spring_promo', 'ds': ['2024-03-15'], ...})
spring_promo = pd.concat([promo1, promo2], ignore_index=True)
```

**흔한 실수**:
- `lower_window`를 양수로 설정 (Prophet은 음수/0을 기대)
- 날짜를 문자열로 두고 datetime 변환 누락

**실무 팁**: 프로모션 사전 광고 효과를 반영하려면 `lower_window`를 음수로 설정하세요. 예: TV 광고 3일 전 시작 → `lower_window=-3`

---

## Q5. 주간 계절성 해석 ⭐⭐

**문제**: 가장 매출이 높은 요일과 낮은 요일을 찾으세요.

In [ ]:
# Q5 정답
# 요일 추출 (0=월, 1=화, ..., 6=일)
forecast_holidays['dayofweek'] = forecast_holidays['ds'].dt.dayofweek

# 요일별 주간 계절성 효과 평균
weekly_effect = forecast_holidays.groupby('dayofweek')['weekly'].mean()

# 요일명 매핑
day_names = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}

# 최고/최저 요일
best_day_idx = weekly_effect.idxmax()
worst_day_idx = weekly_effect.idxmin()

best_day = day_names[best_day_idx]
worst_day = day_names[worst_day_idx]

print(f"가장 매출이 높은 요일: {best_day}요일 ({weekly_effect[best_day_idx]:,.0f}원)")
print(f"가장 매출이 낮은 요일: {worst_day}요일 ({weekly_effect[worst_day_idx]:,.0f}원)")

In [ ]:
# 테스트
assert best_day in ['금', '토', '일'], "주말/금요일이 최고 매출일이어야 합니다"
print("Q5 테스트 통과!")

### 💡 Q5 풀이 설명

**접근 방법**: forecast 결과의 `weekly` 컬럼에 주간 계절성 효과가 저장되어 있습니다.

**핵심 개념**:
- `weekly` 컬럼: 해당 요일의 매출 효과 (원 단위)
- 양수: 평균 대비 매출 상승
- 음수: 평균 대비 매출 하락

**대안 솔루션**:
```python
# Plotly로 시각화하며 확인
fig = px.bar(weekly_effect.reset_index(), x='dayofweek', y='weekly')
fig.show()
```

**흔한 실수**:
- `dt.dayofweek`와 `dt.weekday`가 같은 결과임을 모름 (둘 다 0=월)
- 전체 기간 평균이 아닌 특정 날짜의 값만 확인

**실무 팁**: 주간 패턴 분석 결과를 마케팅 팀에 공유하여 프로모션 시작 요일을 최적화할 수 있습니다.

---

## Q6. Changepoint 설정 ⭐⭐⭐

**문제**: 2023년 10월 1일을 changepoint로 설정한 모델을 만드세요.

In [ ]:
# Q6 정답
model_q6 = Prophet(
    changepoints=['2023-10-01'],  # 수동 changepoint 설정
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False
)

# 학습
model_q6.fit(df_prophet)

# changepoint 확인
print(f"설정된 changepoint: {model_q6.changepoints}")

In [ ]:
# 테스트
assert len(model_q6.changepoints) == 1, "changepoint가 1개여야 합니다"
assert pd.Timestamp('2023-10-01') in list(model_q6.changepoints), "2023-10-01이 changepoint여야 합니다"
print("Q6 테스트 통과!")

### 💡 Q6 풀이 설명

**접근 방법**: `changepoints` 파라미터에 날짜 리스트를 전달하면 수동으로 추세 변화점을 설정할 수 있습니다.

**핵심 개념**:
- Changepoint: 추세의 기울기가 변하는 시점
- 자동 감지: Prophet이 데이터에서 자동으로 찾음
- 수동 설정: 분석가가 비즈니스 이벤트 기반으로 직접 지정

**대안 솔루션**:
```python
# 여러 changepoint 설정
model = Prophet(changepoints=['2023-06-01', '2023-10-01', '2024-01-01'])

# changepoint 범위 제한
model = Prophet(changepoint_range=0.8)  # 데이터의 80%까지만
```

**흔한 실수**:
- 문자열 대신 datetime 객체 전달 (둘 다 가능하지만 문자열이 편리)
- 데이터 범위 밖의 날짜 지정 → 무시됨

**실무 팁**: 사이트 리뉴얼, 대규모 마케팅 캠페인, 경쟁사 진입 등 비즈니스 구조 변화가 있었던 날짜를 changepoint로 설정하면 예측 정확도가 향상됩니다.

---

## Q7. 외부 변수 추가 ⭐⭐⭐

**문제**: 기온을 외부 변수로 추가한 모델을 만드세요.

In [ ]:
# Q7 정답
# 데이터 준비 (ds, y, temperature 포함)
train_with_temp = sales_df[['ds', 'y', 'temperature']].copy()

# 모델 생성
model_q7 = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False
)

# 외부 변수 추가 (fit 전에 호출!)
model_q7.add_regressor('temperature')

# 학습
model_q7.fit(train_with_temp)

print("외부 변수 포함 모델 학습 완료!")
print(f"추가된 regressor: {model_q7.extra_regressors}")

In [ ]:
# 테스트
assert 'temperature' in model_q7.extra_regressors, "temperature가 regressor에 추가되어야 합니다"
print("Q7 테스트 통과!")

### 💡 Q7 풀이 설명

**접근 방법**: `add_regressor()`는 반드시 `fit()` 전에 호출해야 합니다.

**핵심 개념**:
- `add_regressor('col_name')`: 외부 변수 추가
- 학습 데이터에 해당 컬럼이 반드시 존재해야 함
- 예측 시에도 미래 값을 제공해야 함

**대안 솔루션**:
```python
# 여러 regressor 추가
model.add_regressor('temperature')
model.add_regressor('discount_rate')
model.add_regressor('ad_spend')

# standardize 옵션
model.add_regressor('temperature', standardize=True)  # 표준화
```

**흔한 실수**:
- `fit()` 후에 `add_regressor()` 호출 → 에러
- 학습 데이터에 해당 컬럼 누락 → 에러

**실무 팁**: 외부 변수는 미래 값을 알 수 있는 것만 추가하세요. 기상청 예보, 계획된 마케팅 예산, 공휴일 정보 등이 적합합니다.

---

## Q8. 시나리오 분석 ⭐⭐⭐⭐

**문제**: 기온 0°C vs 20°C 시나리오의 7일 총 매출 차이를 계산하세요.

In [ ]:
# Q8 정답
# 미래 7일 데이터프레임 생성
future_q8 = model_regressor.make_future_dataframe(periods=7)

# 기존 데이터의 외부 변수
existing_temp = list(df_with_regressor['temperature'])
existing_discount = list(df_with_regressor['discount_rate'])

# 시나리오 A: 기온 0°C
scenario_a = future_q8.copy()
scenario_a['temperature'] = existing_temp + [0.0] * 7  # 미래 7일 0도
scenario_a['discount_rate'] = existing_discount + [0.05] * 7  # 평상시 할인율
forecast_a = model_regressor.predict(scenario_a)

# 시나리오 B: 기온 20°C
scenario_b = future_q8.copy()
scenario_b['temperature'] = existing_temp + [20.0] * 7  # 미래 7일 20도
scenario_b['discount_rate'] = existing_discount + [0.05] * 7
forecast_b = model_regressor.predict(scenario_b)

# 7일 총 매출 비교
sales_a = forecast_a.tail(7)['yhat'].sum()
sales_b = forecast_b.tail(7)['yhat'].sum()
sales_diff = sales_a - sales_b

print(f"시나리오 A (0°C): {sales_a:,.0f}원")
print(f"시나리오 B (20°C): {sales_b:,.0f}원")
print(f"\n매출 차이: {sales_diff:,.0f}원 (A가 {sales_diff/sales_b*100:.1f}% 높음)")

In [ ]:
# 테스트
assert sales_a > sales_b, "추운 날씨(0°C)가 의류 매출이 높아야 합니다"
print("Q8 테스트 통과!")

### 💡 Q8 풀이 설명

**접근 방법**: 동일한 모델에 다른 외부 변수 값을 주입하여 시나리오별 예측을 수행합니다.

**핵심 개념**:
- 시나리오 분석: "만약 ~라면?" (What-if) 질문에 답하는 방법
- 외부 변수 조작: 미래 기간의 regressor 값만 변경
- 비교 분석: 두 시나리오의 예측값 차이 계산

**대안 솔루션**:
```python
# 여러 시나리오 한번에 비교
scenarios = {'cold': 0, 'mild': 15, 'hot': 30}
results = {}
for name, temp in scenarios.items():
    future['temperature'] = existing_temp + [temp] * 7
    forecast = model.predict(future)
    results[name] = forecast.tail(7)['yhat'].sum()
```

**흔한 실수**:
- 기존 데이터의 외부 변수를 포함하지 않음 → 전체 예측 오류
- 미래 기간의 외부 변수만 변경하지 않고 전체 변경

**실무 팁**: 시나리오 분석은 마케팅 예산 책정, 할인율 결정 등 의사결정에 매우 유용합니다. "할인율을 5% 올리면 매출이 얼마나 증가할까?"에 답할 수 있습니다.

---

## Q9. 교차검증 수행 ⭐⭐⭐⭐

**문제**: 조건에 맞게 교차검증을 수행하고 평균 MAPE를 계산하세요.

In [ ]:
# Q9 정답
# 시계열 교차검증 수행
df_cv_q9 = cross_validation(
    model_with_holidays,
    initial='180 days',   # 첫 학습 기간: 180일 (약 6개월)
    period='14 days',     # Fold 간격: 14일 (2주)
    horizon='14 days'     # 예측 기간: 14일 (2주)
)

print(f"교차검증 결과: {len(df_cv_q9):,}개 예측")

# 성능 지표 계산
df_perf_q9 = performance_metrics(df_cv_q9)

# 평균 MAPE
avg_mape = df_perf_q9['mape'].mean()

print(f"\n평균 MAPE: {avg_mape*100:.2f}%")
print(f"평균 RMSE: {df_perf_q9['rmse'].mean():,.0f}원")

In [ ]:
# 테스트
assert avg_mape < 1.0, "MAPE가 100% 미만이어야 합니다"
assert len(df_cv_q9) > 0, "교차검증 결과가 있어야 합니다"
print("Q9 테스트 통과!")

### 💡 Q9 풀이 설명

**접근 방법**: `cross_validation()` 함수로 Rolling Window CV를 수행합니다.

**핵심 개념**:
- `initial`: 첫 학습 기간 (충분한 패턴 학습 필요)
- `period`: 각 Fold 사이의 간격
- `horizon`: 각 Fold에서 예측하는 기간
- MAPE: Mean Absolute Percentage Error (상대적 오차)

**대안 솔루션**:
```python
# horizon별 MAPE 확인
print(df_perf_q9[['horizon', 'mape']])

# 시각화
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv_q9, metric='mape')
```

**흔한 실수**:
- `initial`이 너무 짧으면 계절성 학습 불가
- `period`가 너무 짧으면 Fold 간 데이터 중복

**실무 팁**: MAPE 10% 이하면 우수, 20% 이하면 양호한 모델로 평가됩니다. 단, MAPE는 실제값이 0에 가까울 때 왜곡될 수 있습니다.

---

## Q10. 비즈니스 리포트 작성 ⭐⭐⭐⭐⭐

**문제**: 마케팅 팀에 제출할 향후 7일 예측 리포트를 작성하세요.

In [ ]:
# Q10 정답
# 향후 7일 예측 데이터 추출
next_7_days = forecast_holidays.tail(7)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()

# 1. 총 예상 매출
total_yhat = next_7_days['yhat'].sum()

# 2. 안전재고 기준
total_upper = next_7_days['yhat_upper'].sum()

# 3. 최고 매출 예상일
best_idx = next_7_days['yhat'].idxmax()
best_day = next_7_days.loc[best_idx]

# 4. 리포트 출력
print("="*60)
print("[마케팅 팀 제출용: 향후 7일 매출 예측 리포트]")
print("="*60)
print(f"\n분석 기간: {next_7_days['ds'].min().date()} ~ {next_7_days['ds'].max().date()}")
print(f"\n1. 총 예상 매출 (점추정): {total_yhat:,.0f}원")
print(f"   - 일 평균: {total_yhat/7:,.0f}원")
print(f"\n2. 안전재고 기준 (95% 신뢰구간 상한): {total_upper:,.0f}원")
print(f"   - 안전 버퍼: +{total_upper - total_yhat:,.0f}원 (+{(total_upper/total_yhat-1)*100:.1f}%)")
print(f"\n3. 최고 매출 예상일: {best_day['ds'].strftime('%Y-%m-%d')} ({best_day['ds'].strftime('%A')})")
print(f"   - 예상 매출: {best_day['yhat']:,.0f}원")
print(f"\n4. 권장 사항:")
print(f"   - 품절 방지를 위해 {total_upper:,.0f}원 상당의 재고 확보 권장")
print(f"   - {best_day['ds'].strftime('%A')}에 프로모션 집중 시 효과 극대화 예상")
print(f"   - 95% 확률로 실제 매출은 {next_7_days['yhat_lower'].sum():,.0f}원 ~ {total_upper:,.0f}원 범위")
print("="*60)

In [ ]:
# 테스트
assert total_yhat > 0, "총 예상 매출이 0보다 커야 합니다"
assert total_upper > total_yhat, "안전재고 기준이 점추정보다 커야 합니다"
print("Q10 테스트 통과!")

### 💡 Q10 풀이 설명

**접근 방법**: Prophet의 예측 결과를 비즈니스 언어로 변환하여 의사결정에 활용합니다.

**핵심 개념**:
- `yhat`: 점추정 (가장 가능성 높은 값)
- `yhat_upper`: 95% 신뢰구간 상한 (품절 방지용)
- `yhat_lower`: 95% 신뢰구간 하한 (과잉재고 방지용)

**대안 솔루션**:
```python
# 시각화 포함 리포트
fig = go.Figure()
fig.add_trace(go.Bar(x=next_7_days['ds'], y=next_7_days['yhat'], name='예상 매출'))
fig.add_trace(go.Scatter(x=next_7_days['ds'], y=next_7_days['yhat_upper'], 
                         mode='lines', name='안전재고 기준', line=dict(dash='dash')))
fig.show()
```

**흔한 실수**:
- 기술적 용어(MAPE, yhat 등)를 그대로 사용
- 불확실성 구간 정보 누락

**실무 팁**: 리포트 작성 시 핵심은 "그래서 어떻게 해야 하는가?"입니다. 숫자만 나열하지 말고 구체적인 액션 아이템(재고 확보량, 프로모션 시점 등)을 제안하세요.

---

## 📊 전체 요약

| 문제 | 난이도 | 핵심 개념 |
|------|--------|----------|
| Q1 | ⭐ | ds, y 컬럼명 변환 |
| Q2 | ⭐ | Prophet 기본 워크플로우 |
| Q3 | ⭐⭐ | 이벤트 효과 컬럼 분석 |
| Q4 | ⭐⭐ | lower_window, upper_window |
| Q5 | ⭐⭐ | 주간 계절성 해석 |
| Q6 | ⭐⭐⭐ | 수동 changepoint 설정 |
| Q7 | ⭐⭐⭐ | add_regressor 사용 |
| Q8 | ⭐⭐⭐⭐ | 시나리오 분석 |
| Q9 | ⭐⭐⭐⭐ | 교차검증 및 MAPE |
| Q10 | ⭐⭐⭐⭐⭐ | 비즈니스 리포트 작성 |